# Before starting

- Copy the protein fasta file of the organism in the directory "/data/annotations/fasta/"
- Be sure you have the trained deep learning models to do the compartment prediction of the proteins in the folder "/bin/compartmentPredictions/deepModels". It can be downloaded from "http://doi.org/10.5281/zenodo.4436488"
- Install the eggNog annotation software: https://github.com/eggnogdb/eggnog-mapper
- Install the python libraries:Tensorflow, keras, biopython, pandas, numpy, cobrapy, Framed (https://github.com/cdanielmachado/framed)

# Functional annotation

download eggnog-mapper from https://github.com/eggnogdb/eggnog-mapper. 

In [4]:
import subprocess

In [7]:
python2_command= "bin/eggnog-mapper-master/emapper.py -i \"data/annotations/fasta/example.fasta\" --database euk --output \"data/annotations/functional/example.fasta\" -m diamond --override --cpu 32"

In [8]:
process = subprocess.Popen(python2_command.split(), stdout=subprocess.PIPE)
output, error = process.communicate() 

In [9]:
output

b'/usr/local/bin/diamond /mnt/home/scsandra/proj/reconstructionPipeline/bin/eggnog-mapper-master\n'

In [10]:
error

# Compartment prediction of the proteins

In [1]:
import pandas as pd
functional_annotation= pd.read_csv("data/annotations/functional/Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein.faa.emapper.annotations",skiprows=3,sep="\t")
functional_annotation = functional_annotation[functional_annotation['EC'].notnull()]
functional_annotation

,#query_name,seed_eggNOG_ortholog,seed_ortholog_evalue,seed_ortholog_score,best_tax_level,Preferred_name,GOs,EC,KEGG_ko,KEGG_Pathway,...,KEGG_rclass,BRITE,KEGG_TC,CAZy,BiGG_Reaction,taxonomic scope,eggNOG OGs,best eggNOG OG,COG Functional cat.,eggNOG free text desc.
0,NP_001018030.1,4932.YCL064C,2.900000e-204,717.6,Saccharomycetaceae,CHA1,"GO:0003674,GO:0003824,GO:0003941,GO:0004794,GO...","4.3.1.17,4.3.1.19",ko:K17989,"ko00260,ko00270,ko00290,ko01100,ko01110,ko0113...",...,"RC00331,RC00418,RC02600","ko00000,ko00001,ko00002,ko01000",NaN,NaN,NaN,Fungi,"38BCJ@33154,3NZW4@4751,3QNQX@4890,3RR5U@4891,3...",NA|NA|NA,E,to Saccharomyces cerevisiae CHA1 (YCL064C)
11,NP_001335748.1,4932.YCR001W,1.600000e-54,218.4,Eukaryota,NaN,"GO:0005575,GO:0005622,GO:0005623,GO:0005737,GO...",4.1.3.1,ko:K01637,"ko00630,ko01100,ko01110,ko01120,ko01200,map006...",...,"RC00311,RC00313","ko00000,ko00001,ko00002,ko01000",NaN,NaN,NaN,Eukaryota,"COG0038@1,KOG0475@2759",NA|NA|NA,P,voltage-gated chloride channel activity
14,NP_009305.1,4932.Q0045,7.600000e-302,1042.3,Saccharomycetaceae,cox1,"GO:0003674,GO:0003824,GO:0004129,GO:0005215,GO...",1.9.3.1,ko:K02256,"ko00190,ko01100,ko04260,ko04714,ko04932,ko0501...",...,NaN,"ko00000,ko00001,ko00002,ko01000,ko03029","3.D.4.11,3.D.4.7,3.D.4.8",NaN,NaN,Fungi,"38JQ7@33154,3NYGG@4751,3QSKE@4890,3RU36@4891,3...",NA|NA|NA,C,electron transport coupled proton transport
27,NP_009319.1,4932.Q0130,2.600000e-30,137.5,Saccharomycetaceae,atp9,"GO:0000276,GO:0003674,GO:0003824,GO:0005215,GO...",1.6.5.3,"ko:K02128,ko:K03880","ko00190,ko01100,ko04714,ko04723,ko05010,ko0501...",...,RC00061,"ko00000,ko00001,ko00002,ko01000,ko03029","3.A.2.1,3.D.1.6",NaN,NaN,Fungi,"3A3YG@33154,3P7BC@4751,3QYEK@4890,3RV97@4891,3...",NA|NA|NA,C,ATP hydrolysis coupled proton transport
37,NP_009339.1,4932.YAL062W,1.500000e-261,908.3,Saccharomycetaceae,GDH3,"GO:0001505,GO:0003674,GO:0003824,GO:0004353,GO...",1.4.1.4,ko:K00262,"ko00220,ko00250,ko00910,ko01100,map00220,map00...",...,"RC00006,RC02799","ko00000,ko00001,ko01000",NaN,NaN,NaN,Fungi,"38GEK@33154,3NVMC@4751,3QKVX@4890,3RRZ0@4891,3...",NA|NA|NA,E,Belongs to the Glu Leu Phe Val dehydrogenases ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5532,NP_116708.1,4932.YFR050C,7.400000e-149,533.1,Saccharomycetaceae,PRE4,"GO:0000502,GO:0005575,GO:0005622,GO:0005623,GO...",3.4.25.1,ko:K02736,"ko03050,map03050",...,NaN,"ko00000,ko00001,ko00002,ko01000,ko01002,ko03051",NaN,NaN,NaN,Fungi,"38G4U@33154,3NUK4@4751,3QJEW@4890,3RSRP@4891,3...",NA|NA|NA,O,Belongs to the peptidase T1B family
5535,NP_116711.3,4932.YFR053C,7.900000e-282,975.7,Saccharomycetaceae,hxk1,"GO:0000166,GO:0001302,GO:0001678,GO:0003674,GO...",2.7.1.1,ko:K00844,"ko00010,ko00051,ko00052,ko00500,ko00520,ko0052...",...,"RC00002,RC00017","ko00000,ko00001,ko00002,ko01000,ko04131",NaN,NaN,"iMM904.YFR053C,iND750.YFR053C",Fungi,"38DFC@33154,3NW32@4751,3QNBF@4890,3RR3P@4891,3...",NA|NA|NA,G,Belongs to the hexokinase family
5536,NP_116714.1,4932.YFR055W,5.500000e-197,693.3,Saccharomycetaceae,NaN,"GO:0000096,GO:0000098,GO:0000724,GO:0000725,GO...",4.4.1.8,ko:K01760,"ko00270,ko00450,ko01100,ko01110,ko01230,map002...",...,"RC00056,RC00069,RC00382,RC00488,RC00710,RC0124...","ko00000,ko00001,ko00002,ko01000",NaN,NaN,NaN,Fungi,"39WRI@33154,3PCP5@4751,3R1A3@4890,3RW6C@4891,3...",NA|NA|NA,E,Cys/Met metabolism PLP-dependent enzyme
5544,NP_602317.3,4932.YFR047C,2.900000e-170,604.4,Saccharomycetaceae,BNA6,"GO:0003674,GO:0003824,GO:0004514,GO:0005575,GO...",2.4.2.19,ko:K00767,"ko00760,ko01100,map00760,map01100",...,RC02877,"ko00000,ko00001,ko00002,ko01000",NaN,NaN,NaN,Fungi,"38D18@33154,3NUQX@4751,3QMBE@4890,3RS6P@4891,3...",NA|NA|NA,H,Involved in the catabolism of quinolinic acid ...


In [2]:
import tempfile
import shutil

dirpath = tempfile.mkdtemp()
# ... do stuff with dirpath
#shutil.rmtree(dirpath)


In [3]:
from Bio import SeqIO
selected=[]
for record in SeqIO.parse("data/annotations/fasta/Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein.faa", "fasta"):
    sel = functional_annotation[functional_annotation["#query_name"]==record.id]
    if len(sel) >0:
        SeqIO.write(record, dirpath+"/"+record.id+".fasta", "fasta")

In [4]:
print(dirpath)

/tmp/tmp_fk_dfss


In [5]:
import sys
sys.path.insert(0, '3rd_party/psipred')
import subprocess

for key, row in functional_annotation.iterrows():
    process = subprocess.Popen(['3rd_party/psipred/runSec.sh', dirpath+"/"+row["#query_name"]],
                         stdout=subprocess.PIPE, 
                         stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    stdout, stderr


In [1]:
dirpath ="/tmp/tmp_fk_dfss"

In [2]:
import sys
sys.path.insert(0, 'bin/compartmentPrediction/')
import predict
predict.predict(dirpath+"/", "Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein")

(1, 500, 3)
(360, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(104, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(534, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(76, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(457, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(417, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(382, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(713, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(269, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(500, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(173, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(1355, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(759, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(837, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(1131, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(399, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(394, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(275, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(306, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(435, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(974, 7)
(1, 500, 7)
(1, 500, 21)
(1, 500, 3)
(687, 7)
(1, 500, 7)


In [ ]:
shutil.rmtree(dirpath)

# Running CarveMe

In [4]:
import sys
sys.path.insert(0, 'bin/')
import CreateModelEggNogPool 

In [5]:
import logging
serf_logger = logging.getLogger('cobra')
serf_logger.setLevel(logging.ERROR)

In [7]:
CreateModelEggNogPool.carveme_pipeline("Scer_v21", "Saccharomyces_cerevisiae_S288C_GCF_000146045.2","data/annotations/functional/Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein.faa.emapper.annotations","data/annotations/fasta/Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein.faa","data/annotations/localization/Saccharomyces_cerevisiae_S288C_GCF_000146045.2_NEWprotein.loc_pred")

Scores calculated
Solution 0
Objective: 808.3812231553678
Status: Unknown

Objective: 0.907626658490266
Status: Optimal

Solution 1
Objective: 808.3812231552508
Status: Unknown

Objective: 0.9073117116060194
Status: Optimal

Solution 2
Objective: 808.3812231551436
Status: Unknown

Objective: 0.5939739728870619
Status: Optimal

Solution 3
Objective: 808.0331473085121
Status: Unknown

Objective: 0.8213937982287794
Status: Optimal

Solution 4
Objective: 808.033147308503
Status: Unknown

Objective: 0.8213937982288066
Status: Optimal

Solution 0
Objective: 922.5048103424522
Status: Unknown

Objective: 8.466720230026676
Status: Optimal

Solution 1
Objective: 921.9217989186573
Status: Unknown

Objective: 7.612555546634004
Status: Optimal

Solution 2
Objective: 921.3685467393694
Status: Unknown

Objective: 7.718138574643925
Status: Optimal

Solution 3
Objective: 921.3685467393104
Status: Unknown

Objective: 7.002076850335239
Status: Optimal

Solution 4
Objective: 921.1638039657746
Status: Unkn

The resulting model contains two numbers corresponding to the objective result of the two MILP problems (models with open and closed bounds)

Remember to create few models and selecte the one with bigger objective results. 

# Analyzing the resulting model

In [9]:
import cobra

In [10]:
model = cobra.io.read_sbml_model("results/Scer_v21-808.2419928165555-921.6655013411128.sbml")

#### Fixing the BIOMASS reaction

In [11]:
from cobra import Model, Reaction, Metabolite
reaction = Reaction('BIOMASS')
reaction.name = model.reactions.BIOMASS.name
reaction.add_metabolites({
    model.metabolites.get_by_id("13BDglcn_1g_c"): -0.253,
    model.metabolites.ash_1g_c: -0.048,
    model.metabolites.atp_c: -60.0,
    model.metabolites.chitin_1g_c: -0.01,
    model.metabolites.dna_1g_c: -0.005,
    model.metabolites.glycogen_1g_c: -0.1,
    model.metabolites.h2o_c: -60.0,
    model.metabolites.mannan_1g_c: -0.07,
    model.metabolites.phospholipid_1g_c: -0.04,
    model.metabolites.protein_1g_c: -0.4,
    model.metabolites.rna_1g_c: -0.058,
    model.metabolites.sterol_1g_r: -0.001,
    model.metabolites.trehalose_1g_c: -0.015,
    model.metabolites.ficytc_c: -0.0001,
    model.metabolites.focytc_c: -0.0001,
    model.metabolites.retn_c: -0.0001,
    model.metabolites.thmtp_c: -0.0001,
    model.metabolites.ribflv_c: -0.0001,
    model.metabolites.btn_c: -0.0001,
    model.metabolites.q6_m: -0.0001,
    model.metabolites.coa_c: -0.0001,
    model.metabolites.thf_c: -0.0001,
    model.metabolites.adp_c: 60.0,
    model.metabolites.biomass_1g_c: 1.0,
    model.metabolites.h_c: 60.0,
    model.metabolites.pi_c: 60.0
})

model.reactions.BIOMASS.remove_from_model()
model.add_reaction(reaction)

In [12]:
model.objective= "BIOMASS"
model.optimize()

,fluxes,reduced_costs
UF00005_C,0.000000,0.000000e+00
UF00014_C,0.000000,0.000000e+00
UF00015_C,0.000000,-1.091292e-03
UF00016_C,0.000000,0.000000e+00
UF00017_C,0.000000,-1.091292e-03
...,...,...
UF01018_M,0.102631,0.000000e+00
UF02331_M,0.000000,-2.775558e-17
UF01998_M,0.102631,0.000000e+00
UF00028_M,0.102631,0.000000e+00


In [13]:
cobra.io.write_sbml_model(model,"results/Scer_v21-808.2419928165555-921.6655013411128.sbml")

### Analyze individual models in the assembly

In [14]:
def extract_single_model_from_ensemble(position, model):
    newModel = model.copy()
    reactiond = {}
    for reaction in model.reactions:
        name = reaction.name.split('_')       
        ensemble=[]
        for n in name[1:]:
            try:
                n = int(n)
                ensemble.append(n) 
            except:
                None
        reactiond[reaction.id]=ensemble       
    
    for reaction in model.reactions:
        presence=reactiond[reaction.id]          
        if presence[position]==0:           
            newModel.remove_reactions([reaction.id])
            
    return newModel 
def get_compartments_rxn(reaction):
    comp =[]
    for metabolite in reaction.metabolites:
        comp.append(metabolite.compartment)
    return list(set(comp))

def get_compartments(model):
    cyt=0
    nucleus=0
    ret=0
    mit=0
    per=0
    for reaction in model.reactions:
        compartments= get_compartments_rxn(reaction)
        for c in compartments:
            if 'c' in c:
                cyt = cyt+1
            elif 'r' in c:
                ret = ret+1
            elif 'x' in c:
                per=per+1
            elif 'n' in c:
                nucleus=nucleus+1
            elif 'm' in c:
                mit =mit+1
    return [cyt, nucleus, ret, mit, per]

In [ ]:
for i in range(25):
    newModel = extract_single_model_from_ensemble(i, model)
    print("Model " +str(i))
    print("Growth: " + str(newModel.optimize().objective_value))
    print("Number of reactions: " + str(len(newModel.reactions)))
    print("Number of compounds: " + str(len(newModel.metabolites)))
    [cyt, nucleus, ret, mit, per]=get_compartments(newModel)
    print("Reactions in cytoplasm: " + str(cyt))
    print("Reactions in endoplasmic reticulum: " + str(ret))
    print("Reactions in mitochondria: " + str(mit))
    print("Reactions in peroxisome: " + str(per))
    print("--------------------------------")

Model 0
Growth: 0.9484703842864195
Number of reactions: 1811
Number of compounds: 1461
Reactions in cytoplasm: 1132
Reactions in endoplasmic reticulum: 65
Reactions in mitochondria: 234
Reactions in peroxisome: 292
--------------------------------
Model 1
Growth: 0.9484703842864297
Number of reactions: 1763
Number of compounds: 1461
Reactions in cytoplasm: 1107
Reactions in endoplasmic reticulum: 65
Reactions in mitochondria: 236
Reactions in peroxisome: 292
--------------------------------
Model 2
Growth: 0.9490855501586049
Number of reactions: 1786
Number of compounds: 1461
Reactions in cytoplasm: 1120
Reactions in endoplasmic reticulum: 65
Reactions in mitochondria: 239
Reactions in peroxisome: 292
--------------------------------
Model 3
Growth: 0.9490855501586178
Number of reactions: 1786
Number of compounds: 1461
Reactions in cytoplasm: 1119
Reactions in endoplasmic reticulum: 65
Reactions in mitochondria: 238
Reactions in peroxisome: 292
--------------------------------
Model 4


In [12]:
for i in range(25):
    newModel = extract_single_model_from_ensemble(i, model)
    print("Model " +str(i))
    print("Growth: " + str(newModel.optimize().objective_value))
    print("Number of reactions: " + str(len(newModel.reactions)))
    print("Number of compounds: " + str(len(newModel.metabolites)))
    [cyt, nucleus, ret, mit, per]=get_compartments(newModel)
    print("Reactions in cytoplasm: " + str(cyt))
    print("Reactions in endoplasmic reticulum: " + str(ret))
    print("Reactions in mitochondria: " + str(mit))
    print("Reactions in peroxisome: " + str(per))
    print("--------------------------------")

Model 0
Growth: 0.9506473077373335
Number of reactions: 1913
Number of compounds: 1493
Reactions in cytoplasm: 1140
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 321
Reactions in peroxisome: 269
--------------------------------
Model 1
Growth: 0.9503148490733468
Number of reactions: 1909
Number of compounds: 1493
Reactions in cytoplasm: 1138
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 320
Reactions in peroxisome: 269
--------------------------------
Model 2
Growth: 0.9503148490733484
Number of reactions: 1904
Number of compounds: 1493
Reactions in cytoplasm: 1134
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 318
Reactions in peroxisome: 269
--------------------------------
Model 3
Growth: 0.9503148490733471
Number of reactions: 1904
Number of compounds: 1493
Reactions in cytoplasm: 1134
Reactions in endoplasmic reticulum: 86
Reactions in mitochondria: 319
Reactions in peroxisome: 268
--------------------------------
Model 4
